# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

import azureml.core
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from sklearn.model_selection import train_test_split

import pandas as pd
from azureml.widgets import RunDetails

import joblib 
import azureml.automl

from azureml.core.model import Model

from azureml.pipeline.core import PipelineData, TrainingOutput

from azureml.pipeline.core import Pipeline

import pickle

from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

from azureml.core.webservice import Webservice, AciWebservice

import logging
import os

import requests
import json

In [2]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'experiment-1'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
experiment-1,quick-starts-ws-134986,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
amlcompute_cluster_name = "capstone"

# Verify that cluster does not exist already

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [5]:
data = pd.read_csv("heart_failure_clinical_records_dataset.csv")

In [6]:
train, test = train_test_split(data, test_size=0.2, random_state=1 )

if "training" not in os.listdir():
    os.mkdir("./training")

if "models" not in os.listdir():
    os.mkdir("./models")
if "outputs" not in os.listdir():
    os.mkdir("./outputs")

train.to_csv("training/train_data.csv", index = False)

data_store = ws.get_default_datastore()
data_store.upload(src_dir = "./training", target_path = 'udacity-project', overwrite = True,  show_progress = True)

Uploading an estimated of 1 files
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_1c92a6540e3d43b7b5c3b95617302b8f

In [7]:
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("udacity-project/train_data.csv"))

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'    
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    
    compute_target = compute_target,
    task = "classification",
    training_data = train_data,
    label_column_name = "DEATH_EVENT",   
    path = project_folder,
    enable_early_stopping= True,
    featurization= 'auto',
    debug_log = "automl_errors.log",
    )

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output= True)
RunDetails(remote_run).show()

Running on remote.
No run_configuration provided, running on capstone with default configuration
Running on remote compute: capstone
Parent Run ID: AutoML_7e5dda8b-129a-40bf-b0eb-0e77ecddb41c

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:     

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTIO

{'runId': 'AutoML_7e5dda8b-129a-40bf-b0eb-0e77ecddb41c',
 'target': 'capstone',
 'status': 'Completed',
 'startTimeUtc': '2021-01-17T05:50:49.243369Z',
 'endTimeUtc': '2021-01-17T06:30:58.179861Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'capstone',
  'AMLSettingsJsonString': '{"path":null,"name":"experiment-1","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-134986","workspace_name":"quick-starts-ws-134986","region":"southcentralus","compute_target":"capstone","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"featuri

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_model, fitted_model = remote_run.get_output()

print(best_model)

Run(Experiment: experiment-1,
Id: AutoML_7e5dda8b-129a-40bf-b0eb-0e77ecddb41c_30,
Type: azureml.scriptrun,
Status: Completed)


In [12]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                subsample=0.3963157894736842,
                                                                                                subsample_for_bin=200000,
                                                                                                subsample_freq=0,
                              

In [20]:
#TODO: Save the best model
best_model.download_file('outputs/model.pkl', 'outputs/model.pkl')
best_model.download_file('outputs/scoring_file_v_1_0_0.py', 'outputs/score_aml.py')
best_model.download_file('automl_driver.py', 'outputs/automl_driver.py')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
model = Model.register(workspace = ws,
                        model_path ="outputs/model.pkl",
                        model_name = "model")


print(model.name)

Registering model model
model


In [22]:
# Define Deployment

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2,
                                              enable_app_insights=True) 


In [23]:
from azureml.core.conda_dependencies import CondaDependencies

azureml_pip_packages = [
    'azureml-interpret', 'azureml-train-automl', 'azureml-defaults'
]

myenv = CondaDependencies.create(conda_packages=['scikit-learn', 'pandas', 'numpy', 'py-xgboost<=0.80'],
                                 pip_packages=azureml_pip_packages,
                                 pin_sdk_version=True)

with open("env.yml","w") as f:
    f.write(myenv.serialize_to_string())

with open("env.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-interpret~=1.19.0
  - azureml-train-automl~=1.19.0
  - azureml-defaults~=1.19.0
- scikit-learn
- pandas
- numpy
- py-xgboost<=0.80
channels:
- anaconda
- conda-forge



In [24]:
# Create the environment

env = Environment.from_conda_specification(name='env',file_path = 'env.yml')

# myenv = Environment(workspace=ws, name="myenv")

inference_config = InferenceConfig(entry_script='./score.py',environment=env)

f = open("envnew.yml", "w")
f.write(env.python.conda_dependencies.serialize_to_string())
f.close()

print("packages", env.python.conda_dependencies.serialize_to_string())

packages # Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-interpret~=1.19.0
  - azureml-train-automl~=1.19.0
  - azureml-defaults~=1.19.0
- scikit-learn
- pandas
- numpy
- py-xgboost<=0.80
channels:
- anaconda
- conda-forge



In [26]:
service_name = 'services'

services = Model.deploy(ws, service_name, [model], inference_config, deployment_config=aci_config)

In [27]:
services.wait_for_deployment(True)
print(services.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [29]:
print("scoring URI: " + services.scoring_uri)

scoring URI: http://ab406a55-6adc-4e32-aa00-1812ebdf7a4d.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [30]:
scoring_uri = 'http://ab406a55-6adc-4e32-aa00-1812ebdf7a4d.southcentralus.azurecontainer.io/score'
headers = {'Content-Type':'application/json'}

test_data = json.dumps({'data':[{
    'age':75,
    'anaemia':0,
    'creatinine_phosphokinase':582,
    'diabetes':0,
    'ejection_fraction':20,
    'high_blood_pressure':1,
    'platelets':265000,
    'serum_creatinine':1.9,
    'serum_sodium':130,
    'sex':1,
    'smoking':0,
    'time':4}
    ]
        })

test_data2 = json.dumps({'data':[{
    'age':40,
    'anaemia':0,
    'creatinine_phosphokinase':321,
    'diabetes':0,
    'ejection_fraction':35,
    'high_blood_pressure':0,
    'platelets':265000,
    'serum_creatinine':1,
    'serum_sodium':130,
    'sex':1,
    'smoking':0,
    'time':198}
    ]
        })


response = requests.post(scoring_uri, data=test_data2, headers=headers)

print("Result:",response.text)

Result: "Must pass 2-d input. shape=()"


TODO: In the cell below, print the logs of the web service and delete the service

In [31]:
print(service.get_logs())

2021-01-17T07:02:53,156659703+00:00 - gunicorn/run 
2021-01-17T07:02:53,159072472+00:00 - rsyslog/run 
2021-01-17T07:02:53,161537944+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_c48771bbe5fa224df5d541af81b23025/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c48771bbe5fa224df5d541af81b23025/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c48771bbe5fa224df5d541af81b23025/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c48771bbe5fa224df5d541af81b23025/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c48771bbe5fa224df5d541af81b23025/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-17T07:02:53,176550596+00:00 - iot-server/run 
EdgeHubC

In [34]:
print(response.status_code)
print(response.elapsed)

service.delete()

200
0:00:00.055716
